<a href="https://colab.research.google.com/github/HenryW0225/Connect-4-Project/blob/main/connect4_ml_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# parameters for generating dataset using negamax algorithm
samples = 5000
depth = 10
min_moves = 0; # for randommly generated positions;
max_moves = 42 - depth;


In [ ]:
# functions to generate dataset using optimized negamax algorithm
import csv, random
from math import inf

W, H = 7, 6
SIZE = W * (H + 1)
COL_MASKS = [(1 << (H + 1)) - 1 << (c * (H + 1)) for c in range(W)]
BOTTOM_MASKS = [1 << (c * (H + 1)) for c in range(W)]
TOP_MASKS = [1 << (H - 1 + c * (H + 1)) for c in range(W)]
FULL_TOP = 0
for m in TOP_MASKS: FULL_TOP |= m
ORDER = [3, 2, 4, 1, 5, 0, 6]
TT = {}

def can_play(mask, c):
    return (mask & TOP_MASKS[c]) == 0

def play(mask, pos, c):
    col = mask & COL_MASKS[c]
    move = (col + BOTTOM_MASKS[c]) & ~col & COL_MASKS[c]
    n = mask | move
    return n, pos ^ move

def winning(p):
    m = p & (p >> (H + 1))
    if m & (m >> (2 * (H + 1))): return True
    m = p & (p >> H)
    if m & (m >> (2 * H)): return True
    m = p & (p >> (H + 2))
    if m & (m >> (2 * (H + 2))): return True
    m = p & (p >> 1)
    return bool(m & (m >> 2))

def key(mask, pos, depth):
    return (mask, pos, depth)

def negamax(mask, pos, depth, alpha, beta):
    k = key(mask, pos, depth)
    if k in TT: return TT[k]
    if winning(pos):
        TT[k] = 1; return 1
    if (mask & FULL_TOP) == FULL_TOP or depth == 0:
        TT[k] = 0; return 0
    a = alpha
    best = -inf
    for c in ORDER:
        if not can_play(mask, c): continue
        m2, p2 = play(mask, pos, c)
        if winning(p2):
            TT[k] = 1; return 1
        v = -negamax(m2, m2 ^ p2, depth - 1, -beta, -a)
        if v > best: best = v
        if best > a: a = best
        if a >= beta or best == 1: break
    out = 1 if best > 0 else (-1 if best < 0 else 0)
    TT[k] = out
    return out

def random_position(min_moves=min_moves, max_moves=max_moves):
    mask = pos = 0
    turn = 0
    n = random.randint(min_moves, max_moves)
    for _ in range(n):
        mv = None
        for c in ORDER:
            if can_play(mask, c): mv = c; break
        if mv is None: break
        c = random.choice([d for d in ORDER if can_play(mask, d)])
        mask, pos = play(mask, pos if turn == 0 else mask ^ pos, c)
        if winning(pos if turn == 0 else mask ^ pos): break
        turn ^= 1
    player_pos = pos if turn == 0 else mask ^ pos
    return mask, player_pos

def to_board(mask, pos):
    b = [0]*(W*H)
    for c in range(W):
        colm = (mask >> (c*(H+1))) & ((1<<(H+1))-1)
        colp = (pos  >> (c*(H+1))) & ((1<<(H+1))-1)
        for r in range(H):
            bit = 1 << r
            if colm & bit:
                b[c*H + r] = 1 if (colp & bit) else -1
    return b

def generate_csv_progress(path, samples=5000, depth=10, include_draw=False, exact_tail=14, every=500):
    with open(path, "w", newline="") as f:
        w = csv.writer(f)
        w.writerow([f"x{i}" for i in range(42)] + ["outcome"])
        written = 0
        wins = losses = draws = 0
        while written < samples:
            if len(TT) > 1_000_000: TT.clear()
            m, p = random_position()
            rem = 42 - m.bit_count()
            d = rem if rem <= exact_tail else depth
            s = negamax(m, p, d, -inf, inf)
            if s != 0 and random.random() < 0.5:
                p = m ^ p
                s = -s
            if s == 0 and draws > min(wins, losses):
                if not include_draw:
                    continue
            if not include_draw and s == 0:
                continue
            lbl = "win" if s == 1 else ("loss" if s == -1 else "draw")
            w.writerow(to_board(m, p) + [lbl])
            written += 1
            if s == 1: wins += 1
            elif s == -1: losses += 1
            else: draws += 1
            if written % every == 0:
                print(f"{written}/{samples} | win={wins} loss={losses} draw={draws}")
    print(f"Done: {samples} rows -> {path}")


In [ ]:
# generating dataset using negamax algorithm
out_path = "/content/connect4_negamax_dataset.csv"
generate_csv_progress(out_path, samples=samples, depth=depth, include_draw=False, exact_tail=14)

# downloading dataset file to computer
from google.colab import files
files.download('/content/connect4_negamax_dataset.csv')


In [ ]:
# uploading dataset file
from google.colab import files
uploaded = files.upload()


In [ ]:
# installing dependencies
!pip install pandas numpy scikit-learn joblib

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib


In [ ]:
import pandas as pd

# using self-generated dataset
url = "connect4_negamax_dataset (1).csv" # different when you create multiple datasets with the same name
df = pd.read_csv(url, header=None)

# Name columns: cell_0–cell_41 and 'outcome'
df.columns = [f"cell_{i}" for i in range(42)] + ["outcome"]

# Enocde outcomes
df["outcome"] = df["outcome"].map({"win": 0, "loss": 1, "draw": 2})



In [ ]:
import pandas as pd

# using UCI Connect 4 dataset (downloaded and uploaded from computer)
!gunzip -c connect-4.data.Z > connect-4.data
url = "connect-4.data"

df = pd.read_csv(url, header=None)

# Name columns: cell_0–cell_41 and 'outcome'
df.columns = [f"cell_{i}" for i in range(42)] + ["outcome"]

# Encode features for UCI dataset: b=0, x=1, o=2;
for i in range(42) :
  df[f"cell_{i}"] = df[f"cell_{i}"].map({"b": 0, "x": 1, "o": -1})

# Enocde outcomes
df["outcome"] = df["outcome"].map({"win": 0, "loss": 1, "draw": 2})

In [ ]:
# verification check
def gravity_ok(row42):
    for c in range(7):
        col = row42[c*6:(c+1)*6]
        for r in range(1,6):
            if col[r] != 0 and col[r-1] == 0:
                return False, (c, r)
    return True, None

ok = True
for _ in range(2000):
    m, p = random_position()
    row = to_board(m, p)
    g, where = gravity_ok(row)
    if not g:
        ok = False
        print("Violation at:", where, "row:", row)
        break
print("format+gravity OK:", ok)

In [ ]:
# checking dataset
df = df.iloc[1:].reset_index(drop=True)
df.head(10)


In [ ]:
# traing ml model using dataset

# seperating features and label of the dataset
X = df.iloc[:, :-1]
y = df['outcome']

# splitting dataset for train section and test section
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# creating random forest classifier model and training to dataset
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=84)
model.fit(X_train, y_train)

# testing dataset and reporting success of model
from sklearn.metrics import classification_report
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


In [ ]:
# compressing model
joblib.dump(model, 'connect4_model.pkl', compress = 5)

In [ ]:
# download ml model
from google.colab import files
files.download('connect4_model.pkl')

In [ ]:
# seperate model testing
simulated_board = [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

probs = model.predict_proba(pd.DataFrame([simulated_board], columns=X.columns))[0]
print(dict(zip(model.classes_, probs)))